In [1]:
import hopsworks

proj = hopsworks.login(engine="python")
fs = proj.get_feature_store()
mr = proj.get_model_registry()

2025-11-11 22:52:19,813 INFO: Initializing external client
2025-11-11 22:52:19,815 INFO: Base URL: https://stagingmain.devnet.hops.works:443
2025-11-11 22:52:20,953 INFO: Python Engine initialized.

Logged in to project, explore it here https://stagingmain.devnet.hops.works:443/p/122


In [2]:
merchant_fg = fs.get_feature_group("merchant_details")
account_fg = fs.get_feature_group("account_details")
bank_fg = fs.get_feature_group("bank_details")
card_fg = fs.get_feature_group("card_details")
cc_trans_aggs_fg = fs.get_feature_group("cc_trans_aggs_fg")
cc_trans_fg = fs.get_feature_group("cc_trans_fg")

2025-11-11 22:57:04,687 WARNING: VersionWarning: No version provided for getting feature group `merchant_details`, defaulting to `1`.

2025-11-11 22:57:05,298 WARNING: VersionWarning: No version provided for getting feature group `account_details`, defaulting to `1`.

2025-11-11 22:57:05,691 WARNING: VersionWarning: No version provided for getting feature group `bank_details`, defaulting to `1`.

2025-11-11 22:57:06,009 WARNING: VersionWarning: No version provided for getting feature group `card_details`, defaulting to `1`.

2025-11-11 22:57:06,409 WARNING: VersionWarning: No version provided for getting feature group `cc_trans_aggs_fg`, defaulting to `1`.

2025-11-11 22:57:06,817 WARNING: VersionWarning: No version provided for getting feature group `cc_trans_fg`, defaulting to `1`.



In [3]:
	
# merchant_id,
# ip_address,
# merchant_country,
# bank_country,
# account_id
# bank_id,
# prev_ip_transaction,
# prev_ts_transaction,


In [8]:
subtree1 = cc_trans_aggs_fg.select_except(['account_id', 'bank_id', 'prev_ip_transaction', 'prev_st_transaction', 'event_time']).join(account_fg.select_features(), on="account_id").join(bank_fg.select(['credit_rating', 'days_since_bank_cr_changed', 'country']), on="bank_id")

2025-11-11 22:58:06,210 INFO: Using ['name', 'address', 'debt_end_prev_month', 'creation_date', 'end_date'] from feature group `account_details` as features for the query. To include primary key and event time use `select_all`.


In [9]:
selection = cc_trans_fg.select_except(['t_id', 'cc_num', 'merchant_id', 'ip_address', 'event_time']).join(merchant_fg.select_features(), prefix="merchant_", on="merchant_id").join(subtree1, on="cc_num")

2025-11-11 22:58:16,265 INFO: Using ['category', 'country', 'cnt_chrgeback_prev_day', 'cnt_chrgeback_prev_week', 'cnt_chrgeback_prev_month'] from feature group `merchant_details` as features for the query. To include primary key and event time use `select_all`.


In [10]:
df = selection.show(100)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (18.34s) 


In [11]:
df

,amount,card_present,haversine_distance,time_since_last_trans,days_to_card_expiry,is_fraud,merchant_category,merchant_country,merchant_cnt_chrgeback_prev_day,merchant_cnt_chrgeback_prev_week,...,prev_card_present,prev_ts_transaction,name,address,debt_end_prev_month,creation_date,end_date,credit_rating,days_since_bank_cr_changed,country
0,30.09,True,True,63432,0,False,Home Improvement,Argentina,4.11,25.81,...,True,2025-09-08 00:03:39,None,None,NaN,NaT,NaT,NaN,NaN,None
1,39.29,True,False,6567,0,False,Fashion,Australia,3.51,20.50,...,True,2025-09-08 17:40:51,None,None,NaN,NaT,NaT,NaN,NaN,None
2,5.92,True,False,24351,0,False,None,None,NaN,NaN,...,True,2025-09-08 19:30:18,None,None,NaN,NaT,NaT,NaN,NaN,None
3,11.59,False,True,70966,0,False,Pharmacy,China,1.92,12.03,...,False,2025-09-09 02:16:09,None,None,NaN,NaT,NaT,NaN,NaN,None
4,30.43,False,True,36251,0,False,Gas Stations,Spain,4.51,27.19,...,False,2025-09-09 21:58:55,None,None,NaN,NaT,NaT,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,33.60,False,True,37754,0,False,Insurance,Nigeria,4.00,24.97,...,False,2025-09-13 21:33:44,None,None,NaN,NaT,NaT,NaN,NaN,None
96,109.77,False,True,262923,0,False,Books,Brazil,1.17,8.84,...,False,2025-09-14 08:02:58,None,None,NaN,NaT,NaT,NaN,NaN,None
97,113.42,False,True,3755,0,False,Groceries,Nigeria,0.10,0.61,...,False,2025-09-17 09:05:01,None,None,NaN,NaT,NaT,NaN,NaN,None
98,17.11,False,True,159753,0,False,Entertainment,Australia,0.83,6.16,...,False,2025-09-17 10:07:36,None,None,NaN,NaT,NaT,NaN,NaN,None


In [ ]:
fv = fs.get_or_create_feature_view(name="cc_fraud_fv", 
                                   version=1, 
                                   description="features for a credit card fraud prediction model",
                                   query=selection,
                                   labels=['is_fraud']
                                  )